# Modelação de Time Series - LSTM

## Imports

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## Leitura

In [ ]:
braga_time_series = pd.read_csv('../model/datasets/FINALDATASET.csv')

# merge date and time columns into a datetime column
braga_time_series['datetime'] = pd.to_datetime(braga_time_series['date'] + ' ' + braga_time_series['time'])
braga_time_series.set_index('datetime', inplace=True)

# drop the time column
braga_time_series.drop('time', axis=1, inplace=True)

# replace missing values with the median
missing_columns = ['pm10', 'pm25', 'pm1', 'no2']

for column in missing_columns:
    braga_time_series[column] = braga_time_series[column].fillna(braga_time_series[column].median())

# remove NaT values from datetime index
braga_time_series = braga_time_series[braga_time_series.index.notnull()]

# drop the date column
braga_time_series.drop('date', axis=1, inplace=True)

braga_time_series.head()

## Modelação

In [ ]:
train_size = int(len(braga_time_series) * 0.8)  # 80% for training
train_data = braga_time_series[:train_size]
test_data = braga_time_series[train_size:]

input_dim = 4  # number of features
output_dim = 1  # number of targets
input_sequence_length = 24  # number of time steps in each sample

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(input_sequence_length, input_dim)))
model.add(Dense(output_dim))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, batch_size=32, epochs=10)

loss = model.evaluate(X_test, y_test)

predictions = model.predict(new_data)
